In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score

In [28]:
df=pd.read_csv('/kaggle/input/travel-insurance/travel insurance.csv')
df.head()

In [29]:
df.shape

In [30]:
df.rename(columns={ 'Agency Type':'Agency_Type', 'Distribution Channel':'Distribution_Channel', 'Product Name':'Product_Name','Net Sales':'Net_Sales', 'Commision (in value)':'Commision'},inplace=True)

In [31]:
df.info()

In [32]:
df.isnull().sum()

In [33]:
df=df.drop(['Gender'],axis=1)
df.head()
#we remove gender coz there are many null values

In [34]:
df.hist(figsize=(20,20))

In [35]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
df['Agency']=le.fit_transform(df['Agency'])
df['Agency_Type']= le.fit_transform(df['Agency_Type'])
df['Distribution_Channel']= le.fit_transform(df['Distribution_Channel'])
df['Product_Name']= le.fit_transform(df['Product_Name'])
df['Claim']= le.fit_transform(df['Claim'])
df['Destination']= le.fit_transform(df['Destination'])

In [36]:
df.Claim.value_counts().plot(kind='pie')

In [37]:
col=["Agency", "Agency_Type", "Distribution_Channel","Product_Name","Duration","Destination","Net_Sales","Commision","Age","Claim"]
df=df.reindex(columns=col)

In [38]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(),annot=True,cmap='Blues')
#all values are less than 0.7 so we dont do anything

In [39]:
y=df.iloc[:,[-1]]
x=df.drop("Claim",axis=1)

In [40]:
x

In [41]:
y

In [42]:
sm = SMOTE(random_state=42)
xsm, ysm = sm.fit_resample(x, y)
ysm.value_counts().plot(kind='pie')

In [45]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
xn=pd.DataFrame()
col=xsm.columns
xn[col]=mms.fit_transform(xsm[col])
xn.head()


In [46]:
xt,xtest,yt,ytest=train_test_split(xn,ysm,test_size=0.25,random_state=42)

In [47]:
mod=RandomForestClassifier(random_state=42)
mod.fit(xt,yt)
pred=mod.predict(xtest)

In [48]:
print(accuracy_score(ytest,pred))

In [51]:
mod.score(xt,yt)

In [52]:
mod.score(xtest,ytest)

In [49]:
sns.heatmap(confusion_matrix(ytest,pred),annot=True,cmap='Blues')

In [50]:
f1_score(ytest,pred,average='micro')